In [36]:
# 载入python库
import pandas as pd
import networkx as nx
import xlrd, openpyxl
import re
import os
from copy import deepcopy

In [2]:
def fliter_none(issue):
    pattern = re.compile('\d{6}.[A-Z]{2}')
    try:
        id_stock = pattern.search(issue)
        if id_stock is not None :
            return True
        else:
            return False
    except:
        return False 

In [54]:
def generate_dict(dataframe):
    fund_network = {}
    for _, row in dataframe.iterrows():
        fund, stock = row
        if fund in fund_network:
            fund_network[fund].append(stock)
        else:    
            fund_network[fund] = []
            fund_network[fund].append(stock)
    return fund_network

In [61]:
def generate_network_dataframe(fund_network,fund_network_copy):
    network_dataframe = {'source':[],'target':[],'stock':[],'weight':[]}
    for key1, stock_list1 in fund_network.items():
        del fund_network_copy[key1]
        for key2, stock_list2 in fund_network_copy.items():
            share_element = [x for x in stock_list1 if x in stock_list2]
            if len(share_element)>0:
                network_dataframe['source'].append(key1)
                network_dataframe['target'].append(key2)
                network_dataframe['weight'].append(len(share_element))
                network_dataframe['stock'].append('*'.join(share_element))
    return pd.DataFrame(network_dataframe)

In [57]:
def xlsx_csv_fund(xlsx_filepath, csv_filepath, fund_filepath):
    list_dir =  [os.path.splitext(dir) for dir in os.listdir(xlsx_filepath)]
    for tuple_dir in list_dir:
        date_name, format_name = tuple_dir
        dataframe = pd.read_excel(xlsx_filepath+'/'+date_name+format_name)
        data_csv = dataframe[dataframe['代码'].apply(fliter_none)][['代码','名称','股票代码','股票简称']]
        data_csv = data_csv.rename(columns = {'代码':'code','名称':'source','股票代码':'stock_code','股票简称':'target'})
        csv_name = ('_').join(date_name.split('-'))
        data_csv[['source','target']].to_csv(csv_filepath+'/'+csv_name+'.csv', encoding='utf_8_sig', index=False)
        dataframe_fund = data_csv[['source','target']]
        fund_network = generate_dict(dataframe_fund)
        fund_network_copy = deepcopy(fund_network)
        network_dataframe = generate_network_dataframe(fund_network,fund_network_copy)
        network_dataframe.to_csv(fund_filepath+'/'+csv_name+'.csv', encoding='utf_8_sig', index=False)

In [58]:
xlsx_filepath = './Dataset/StockData'
csv_filepath = './Dataset/StockData_csv'
fund_filepath = './Dataset/StockData_fund'

In [62]:
xlsx_csv_fund(xlsx_filepath, csv_filepath, fund_filepath)

In [13]:
dataframe

,source,target
0,嘉实研究阿尔法,中国平安
1,嘉实研究阿尔法,招商银行
2,嘉实研究阿尔法,贵州茅台
3,嘉实研究阿尔法,新华保险
4,嘉实研究阿尔法,立讯精密
...,...,...
3969,中金新锐A,中微公司
3970,中金新锐A,兴发集团
3971,中金新锐A,宇信科技
3972,中金新锐A,石英股份
